In [1]:
!pip install torch transformers torchvision requests pandas pillow datasets
!pip install faiss-cpu
!pip install chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 915.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!pip install kagglehub kaggle

In [8]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("raddar/chest-xrays-indiana-university")

print("Path to dataset files:", path)

100%|██████████| 13.2G/13.2G [02:35<00:00, 91.0MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/raddar/chest-xrays-indiana-university/versions/2


In [9]:
!rm -r /root/.cache/kagglehub/datasets/financekim

In [17]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import torch
import faiss
from transformers import FlavaProcessor, FlavaModel
reports_df = pd.read_csv("/root/.cache/kagglehub/datasets/raddar/chest-xrays-indiana-university/versions/2/indiana_reports.csv")
projections_df = pd.read_csv("/root/.cache/kagglehub/datasets/raddar/chest-xrays-indiana-university/versions/2/indiana_projections.csv")
df = reports_df.merge(projections_df, on="uid")
print("Dataset Overview:")
print(df.head())
model_name = "facebook/flava-full"
processor = FlavaProcessor.from_pretrained(model_name)
model = FlavaModel.from_pretrained(model_name)
model.eval()
def extract_multimodal_features(image_path, text):
    """Extract consistent combined multimodal features from FLAVA."""
    image = Image.open(image_path).convert("RGB")
    inputs = processor(text=[text], images=image, return_tensors="pt", padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        features = outputs.multimodal_embeddings[:, 0, :].squeeze(0).numpy()
    return features
sample_image_path = os.path.join("/root/.cache/kagglehub/datasets/raddar/chest-xrays-indiana-university/versions/2/images/images_normalized", df["filename"].iloc[0])
sample_text = df["findings"].iloc[0]
sample_features = extract_multimodal_features(sample_image_path, sample_text)
dim = sample_features.shape[0]
index = faiss.IndexFlatL2(dim)
report_embeddings = []
case_texts = []
processed_count = 0
max_images = 50
for i, row in df.iterrows():
    if processed_count >= max_images:
        break
    image_path = os.path.join("/root/.cache/kagglehub/datasets/raddar/chest-xrays-indiana-university/versions/2/images/images_normalized", row["filename"])
    case_text = row["findings"]

    if pd.isnull(case_text) or not isinstance(case_text, str) or not os.path.exists(image_path):
        continue

    if "impressions" in df.columns and pd.notna(row["impressions"]):
        case_text += " " + row["impressions"]
    combined_features = extract_multimodal_features(image_path, case_text)
    if combined_features.shape[0] == dim:
        report_embeddings.append(combined_features)
        case_texts.append(case_text)
        processed_count += 1
    else:
        print(f"Skipping entry with mismatched dimensions: {combined_features.shape}")
if report_embeddings:
    index.add(np.vstack(report_embeddings).astype(np.float32))
    print(f"FAISS index built with {len(report_embeddings)} embeddings.")
else:
    print("No valid embeddings found to build the FAISS index.")
def retrieve_similar_cases(query_text, query_image_path, top_k=3):
    """Retrieve similar cases from the RAG system."""
    query_features = extract_multimodal_features(query_image_path, query_text)

    if query_features.shape[0] == dim:
        distances, indices = index.search(query_features.reshape(1, -1), top_k)
        return [case_texts[i] for i in indices[0] if i < len(case_texts)]
    else:
        print("Query features dimension mismatch.")
        return []
query_text = "Patient has pneumonia with fluid accumulation."
query_image_path = os.path.join("/root/.cache/kagglehub/datasets/raddar/chest-xrays-indiana-university/versions/2/images/images_normalized", df["filename"].iloc[1])
retrieved_cases = retrieve_similar_cases(query_text, query_image_path)
print("\nRetrieved Similar Cases:")
for case in retrieved_cases:
    print("-", case)

Dataset Overview:
   uid                                               MeSH  \
0    1                                             normal   
1    1                                             normal   
2    2  Cardiomegaly/borderline;Pulmonary Artery/enlarged   
3    2  Cardiomegaly/borderline;Pulmonary Artery/enlarged   
4    3                                             normal   

                        Problems                                image  \
0                         normal            Xray Chest PA and Lateral   
1                         normal            Xray Chest PA and Lateral   
2  Cardiomegaly;Pulmonary Artery  Chest, 2 views, frontal and lateral   
3  Cardiomegaly;Pulmonary Artery  Chest, 2 views, frontal and lateral   
4                         normal            Xray Chest PA and Lateral   

                                          indication comparison  \
0                                   Positive TB test      None.   
1                                   Positi

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1113: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


FAISS index built with 50 embeddings.

Retrieved Similar Cases:
- The cardiac contours are normal. The lungs are clear. Thoracic spondylosis.
- The cardiac contours are normal. The lungs are clear. Thoracic spondylosis.
- Cardiomediastinal silhouette and pulmonary vasculature are within normal limits. Lungs are clear. No pneumothorax or pleural effusion. No acute osseous findings.


In [ ]:
query_text = "Patient has pneumonia with fluid accumulation."
query_image_path = os.path.join("/root/.cache/kagglehub/datasets/raddar/chest-xrays-indiana-university/versions/2/images/images_normalized", df["filename"].iloc[1])
retrieved_cases = retrieve_similar_cases(query_text, query_image_path)
print("\nRetrieved Similar Cases:")
for case in retrieved_cases:
    print("-", case)